In [ ]:
import sys
import os
IMATOOLS_DIR = os.getcwd()+'/..'
sys.path.insert(1, IMATOOLS_DIR)

import pandas as pd 
import numpy as np


In [ ]:
from imatools.common import ioutils as iou 

# Locations of hard drive based on platform
dirdic={'macOS'   : '/Volumes/sandisk', 
        'Linux'   : '/media/jsl19/sandisk' , 
        'Windows' : 'D:/'}

p2f = iou.fullfile(dirdic[iou.chooseplatform()], '09-dnav_vs_inav/umc/local')
name = 'spatial_correspondence.csv'

df = pd.read_csv(iou.fullfile(p2f, name), skipinitialspace=True)


In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

x=np.linspace(0, 0.7, 10)
y=x

sns.set_palette(palette='colorblind')
sns.scatterplot(data=df, x='fibrosis_d', y='fibrosis_i', hue='th_inav')

plt.plot(x,y)


In [ ]:

sns.set_palette(palette='colorblind')
sns.boxplot(x=df['th_inav'], y=(df['fibrosis_d']-df['fibrosis_i'])**2)


In [ ]:
sns.boxplot(data=df, x='th_inav', y='perf_name')

In [ ]:
df['fibrosis_d']-df['fibrosis_i']
